In [9]:
#importing required packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings

warnings.filterwarnings('ignore')

In [10]:
#Read Dataset
credit_data = pd.read_excel(r"C:\Users\samue\OneDrive\Documents\Bobby\Datasets\credit_scoring.xlsx")

In [11]:
credit_data.head()

,TARGET,ID,DerogCnt,CollectCnt,BanruptcyInd,InqCnt06,InqTimeLast,InqFinanceCnt24,TLTimeFirst,TLTimeLast,...,TL50UtilCnt,TLBalHCPct,TLSatPct,TLDel3060Cnt24,TLDel90Cnt24,TLDel60CntAll,TLOpenPct,TLBadDerogCnt,TLDel60Cnt24,TLOpen24Pct
0,1,582,3,3,0,4,0.0,5,117,27,...,3.0,0.9179,0.2083,2,3,7,0.2083,4,4,0.0000
1,1,662,15,9,0,3,1.0,3,14,14,...,1.0,0.8000,0.0000,0,0,0,1.0000,12,0,1.0000
2,1,805,0,0,0,1,5.0,1,354,7,...,5.0,0.3552,0.6538,0,1,1,0.7308,1,1,0.5263
3,1,1175,8,5,0,6,1.0,10,16,4,...,3.0,0.9127,0.2500,1,1,1,0.7500,7,1,1.3333
4,1,1373,3,1,0,9,0.0,8,130,52,...,1.0,1.2511,0.0000,0,1,4,0.1429,3,1,0.0000


### Exploration phase

In [12]:
#Let's see what columns we have
credit_data.columns

Index(['TARGET', 'ID', 'DerogCnt', 'CollectCnt', 'BanruptcyInd', 'InqCnt06',
       'InqTimeLast', 'InqFinanceCnt24', 'TLTimeFirst', 'TLTimeLast',
       'TLCnt03', 'TLCnt12', 'TLCnt24', 'TLCnt', 'TLSum', 'TLMaxSum',
       'TLSatCnt', 'TLDel60Cnt', 'TLBadCnt24', 'TL75UtilCnt', 'TL50UtilCnt',
       'TLBalHCPct', 'TLSatPct', 'TLDel3060Cnt24', 'TLDel90Cnt24',
       'TLDel60CntAll', 'TLOpenPct', 'TLBadDerogCnt', 'TLDel60Cnt24',
       'TLOpen24Pct'],
      dtype='object')

In [13]:
#check for consistent data types
credit_data.dtypes

TARGET               int64
ID                   int64
DerogCnt             int64
CollectCnt           int64
BanruptcyInd         int64
InqCnt06             int64
InqTimeLast        float64
InqFinanceCnt24      int64
TLTimeFirst          int64
TLTimeLast           int64
TLCnt03              int64
TLCnt12              int64
TLCnt24              int64
TLCnt              float64
TLSum              float64
TLMaxSum           float64
TLSatCnt           float64
TLDel60Cnt           int64
TLBadCnt24           int64
TL75UtilCnt        float64
TL50UtilCnt        float64
TLBalHCPct         float64
TLSatPct           float64
TLDel3060Cnt24       int64
TLDel90Cnt24         int64
TLDel60CntAll        int64
TLOpenPct          float64
TLBadDerogCnt        int64
TLDel60Cnt24         int64
TLOpen24Pct        float64
dtype: object

In [15]:
#check dataset for duplicates
credit_data.duplicated().sum()

0

#### No duplicates. Great! Let's see if we have missing data

In [25]:
credit_data.isnull().sum()

TARGET               0
ID                   0
DerogCnt             0
CollectCnt           0
BanruptcyInd         0
InqCnt06             0
InqTimeLast        188
InqFinanceCnt24      0
TLTimeFirst          0
TLTimeLast           0
TLCnt03              0
TLCnt12              0
TLCnt24              0
TLCnt                3
TLSum               40
TLMaxSum            40
TLSatCnt             4
TLDel60Cnt           0
TLBadCnt24           0
TL75UtilCnt         99
TL50UtilCnt         99
TLBalHCPct          41
TLSatPct             4
TLDel3060Cnt24       0
TLDel90Cnt24         0
TLDel60CntAll        0
TLOpenPct            3
TLBadDerogCnt        0
TLDel60Cnt24         0
TLOpen24Pct          3
dtype: int64

We can use the mean inputation method for our missing values. While this is not the best approach because further exploration could lead us to more information on whether our data is missing completely at random or not at random, thereby allowing us implement the best imputation method that mirrors the exact data generating process for our dataset

In [53]:
#before we make changes to our dataset, let's make a copy so we preserve the original
credit_data_X = credit_data.copy()

#Mean imputation
credit_data_X.fillna(credit_data_X.mean(), inplace=True)


In [54]:
#check status of missing data
credit_data_X.isnull().sum()

TARGET             0
ID                 0
DerogCnt           0
CollectCnt         0
BanruptcyInd       0
InqCnt06           0
InqTimeLast        0
InqFinanceCnt24    0
TLTimeFirst        0
TLTimeLast         0
TLCnt03            0
TLCnt12            0
TLCnt24            0
TLCnt              0
TLSum              0
TLMaxSum           0
TLSatCnt           0
TLDel60Cnt         0
TLBadCnt24         0
TL75UtilCnt        0
TL50UtilCnt        0
TLBalHCPct         0
TLSatPct           0
TLDel3060Cnt24     0
TLDel90Cnt24       0
TLDel60CntAll      0
TLOpenPct          0
TLBadDerogCnt      0
TLDel60Cnt24       0
TLOpen24Pct        0
dtype: int64

In [27]:
#let's see how many features we have in our dataset
credit_data.shape

(3000, 30)

We have 28 features, a unique identifier and a target variable (A column that tells us what the outcome of each observation was; either negative, denoted by `1` or positive, denoted by `0`)

We may not require all 28 features to help our model understand the true data generating process of our dataset but we sure would like to have as little features as possible with all the information required to make our model close to perfection, which means reducing the noise in our data by removing unnecessary features that could lead to overfitting. Let's see how we can use PCA to achieve that.

#### Principal Component Analysis

Principal Component Analysis (PCA) is a technique used in Python and machine learning to reduce the dimensionality of high-dimensional data while preserving the most important information.

In [55]:

#Drop ID column
credit_data_X.drop(["ID"], axis=1, inplace=True)
credit_data_X.head()

,TARGET,DerogCnt,CollectCnt,BanruptcyInd,InqCnt06,InqTimeLast,InqFinanceCnt24,TLTimeFirst,TLTimeLast,TLCnt03,...,TL50UtilCnt,TLBalHCPct,TLSatPct,TLDel3060Cnt24,TLDel90Cnt24,TLDel60CntAll,TLOpenPct,TLBadDerogCnt,TLDel60Cnt24,TLOpen24Pct
0,1,3,3,0,4,0.0,5,117,27,0,...,3.0,0.9179,0.2083,2,3,7,0.2083,4,4,0.0000
1,1,15,9,0,3,1.0,3,14,14,0,...,1.0,0.8000,0.0000,0,0,0,1.0000,12,0,1.0000
2,1,0,0,0,1,5.0,1,354,7,0,...,5.0,0.3552,0.6538,0,1,1,0.7308,1,1,0.5263
3,1,8,5,0,6,1.0,10,16,4,0,...,3.0,0.9127,0.2500,1,1,1,0.7500,7,1,1.3333
4,1,3,1,0,9,0.0,8,130,52,0,...,1.0,1.2511,0.0000,0,1,4,0.1429,3,1,0.0000


In [56]:
#We should set our target variable aside so it's not affected by the dimensionality reduction exercise.

y = credit_data_X['TARGET']
X = credit_data_X.drop(['TARGET'], axis=1)

In [57]:
X.head()
y.head()

0    1
1    1
2    1
3    1
4    1
Name: TARGET, dtype: int64

In [59]:
#Standardize our dataset features into a unit scale because the PCA is affected by variance in scale
X_scaled = StandardScaler().fit_transform(X)

In [62]:
#Examine scaled data
X_scaled

array([[ 0.57487821,  0.99167442, -0.42556123, ...,  1.05324168,
         1.62345171, -1.17598354],
       [ 4.96885178,  3.76817768, -0.42556123, ...,  4.30524222,
        -0.59160463,  0.90828415],
       [-0.52361518, -0.39657722, -0.42556123, ..., -0.16625853,
        -0.03784055, -0.07903346],
       ...,
       [-0.15745072,  0.06617333, -0.42556123, ..., -0.16625853,
        -0.59160463,  0.38721723],
       [ 0.94104267,  1.45442496, -0.42556123, ...,  1.05324168,
        -0.03784055,  0.90828415],
       [ 0.20871374,  0.06617333,  2.34983811, ...,  0.64674161,
         0.51592354, -1.17598354]])

In [63]:
#implementing PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=15)
principal_components = pca.fit_transform(X_scaled)

In [65]:
#Examine the shape of our adjusted dataset
principal_components.shape

(3000, 15)

In [71]:
pca.explained_variance_

array([6.66425375, 4.61379602, 2.99438122, 1.83139346, 1.42768139,
       1.354543  , 1.30094313, 1.20383124, 0.86959617, 0.79865353,
       0.75985139, 0.70920127, 0.63542957, 0.55885989, 0.41959176])

In [66]:
pca_credit_data = pd.DataFrame(principal_components)

In [67]:
pca_credit_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,3.040505,1.521939,-0.847817,1.055572,-1.106023,1.367968,0.025781,1.048080,-0.009517,-0.278106,0.041235,-1.040712,0.234824,-0.452742,0.863569
1,3.564049,-1.018554,3.595808,6.387023,0.800924,-0.103095,0.063304,0.324074,0.804665,-0.465628,-0.723255,-0.825966,-0.686207,0.529644,0.253687
2,-2.146034,0.564004,-1.498970,0.041820,0.294629,-1.557127,-1.284772,-0.957012,-0.404103,-0.216746,0.045486,1.632545,0.606865,0.332033,-0.014521
3,1.836940,0.222763,3.679389,2.710808,0.582188,0.873894,-0.031269,1.202462,-0.457884,-0.082970,-0.324209,-0.288982,-0.600425,0.486365,0.019573
4,3.737444,-0.361824,0.543317,-0.121210,-1.302755,2.841647,1.233014,0.055270,0.362156,-0.602189,2.093677,-1.106630,0.711580,-0.363570,1.553116


In [77]:
#Map target variable to principal components
pca_credit_data['Target'] = y

pca_credit_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Target
0,3.040505,1.521939,-0.847817,1.055572,-1.106023,1.367968,0.025781,1.048080,-0.009517,-0.278106,0.041235,-1.040712,0.234824,-0.452742,0.863569,1
1,3.564049,-1.018554,3.595808,6.387023,0.800924,-0.103095,0.063304,0.324074,0.804665,-0.465628,-0.723255,-0.825966,-0.686207,0.529644,0.253687,1
2,-2.146034,0.564004,-1.498970,0.041820,0.294629,-1.557127,-1.284772,-0.957012,-0.404103,-0.216746,0.045486,1.632545,0.606865,0.332033,-0.014521,1
3,1.836940,0.222763,3.679389,2.710808,0.582188,0.873894,-0.031269,1.202462,-0.457884,-0.082970,-0.324209,-0.288982,-0.600425,0.486365,0.019573,1
4,3.737444,-0.361824,0.543317,-0.121210,-1.302755,2.841647,1.233014,0.055270,0.362156,-0.602189,2.093677,-1.106630,0.711580,-0.363570,1.553116,1


#### Now let's teach our model how this data was generated

We'll be using Logistic Regression, a regression model whose output can also be used as a threshold to classify binary outcomes like we are doing now.

In [78]:
#Split principal components into training and test data
y = pca_credit_data['Target']
X = pca_credit_data.drop(['Target'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [79]:
X_train.shape

(2400, 15)

In [82]:
y.shape

(3000,)

In [83]:
#initialize our Risk Model
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

Check our Risk Model performance

In [84]:
print(confusion_matrix(y_test, y_pred))

[[484  14]
 [ 89  13]]


In [85]:
print(accuracy_score(y_test, y_pred))

0.8283333333333334


In [86]:
#Use our Risk Model to predict outcomes for our test data
predictions = classifier.predict_proba(X_test)
predictions

array([[0.04199189, 0.95800811],
       [0.91619513, 0.08380487],
       [0.8167344 , 0.1832656 ],
       ...,
       [0.96589682, 0.03410318],
       [0.36519825, 0.63480175],
       [0.90802476, 0.09197524]])

We can see an `82.5%` accuracy from our model. This is a fair outcome but we always want our model to be better, especially in situations where the margin for prediction error is very low. You neither want to grant a bad loan nor reject a good loan application.

Let's try splitting our data with `Cross Validation` as a first step to model improvement.

In [109]:
from numpy import std, mean
from sklearn.model_selection import KFold, cross_val_score, LeaveOneOut

cv = KFold(n_splits=10, random_state=0, shuffle=True)
classifier2 = LogisticRegression()
# evaluate model
scores = cross_val_score(classifier2, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print(f'Accuracy: {round(mean(scores),2)}')

Accuracy: 0.84


A bit of improvement in our model accuracy.

In [110]:

classifier2.fit(X_train, y_train)
y_pred_2 = classifier.predict(X_test)
predictions_2 = classifier2.predict_proba(X_test)
predictions_2

array([[0.04199189, 0.95800811],
       [0.91619513, 0.08380487],
       [0.8167344 , 0.1832656 ],
       ...,
       [0.96589682, 0.03410318],
       [0.36519825, 0.63480175],
       [0.90802476, 0.09197524]])

In [111]:
print(confusion_matrix(y_test, y_pred_2))

[[484  14]
 [ 89  13]]


### Data Cleaning